## Importando bibliotecas necessárias

In [1]:
import time
import os
from pathlib import Path
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv


## Variáveis de ambiente

In [2]:
load_dotenv()

llm = AzureChatOpenAI(
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_type=os.getenv("OPENAI_API_TYPE"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    temperature=0.0,
)
embeddings = OpenAIEmbeddings(
    deployment=os.getenv("EMBEDDING_DEPLOYMENT_NAME"),
    chunk_size=15
)

c:\Users\BlueShift\Documents\Projects\helpy_bot\mybot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
c:\Users\BlueShift\Documents\Projects\helpy_bot\mybot\Lib\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://blue-openai-academy.openai.azure.com/ to https://blue-openai-academy.openai.azure.com/openai.
  warnings.warn(
c:\Users\BlueShift\Documents\Projects\helpy_bot\mybot\Lib\sit

## Recebendo dataframe

In [3]:
path_project = Path.cwd().resolve().parents[0]
data_dir = path_project / 'doc' / 'dataframe_apostila.csv'
apostila_dataframe = pd.read_csv(data_dir)

display(apostila_dataframe.head())

,Capitulo,Seção,Contexto
0,CAPÍTULO 1 COMO APRENDER PYTHON,1.2 SOBRE OS EXERCÍCIOS,1 COMO APRENDER PYTHON \n1
1,CAPÍTULO 1 COMO APRENDER PYTHON,1.3 TIRANDO DÚVIDAS E INDO ALÉM,"Para tirar dúvidas de exercícios, ou de Python..."
2,CAPÍTULO 2 O QUE É PYTHON,2.2 BREVE HISTÓRIA,Python foi criada em 1990 por Guido Van Rossum...
3,CAPÍTULO 2 O QUE É PYTHON,2.3 INTERPRETADOR,Você provavelmente já ouviu ou leu em algum lu...
4,CAPÍTULO 2 O QUE É PYTHON,2.4 QUAL VERSÃO UTILIZAR?,"Para quem está começando, a primeira dúvida na..."


## Criando metadados

In [4]:
document_list = []

for index, row in apostila_dataframe.iterrows():
    paragraph = Document(
        page_content=row["Contexto"],
        metadata={
            "Chapter": row["Capitulo"],
            "Session": row["Seção"],
        },
    )
    document_list.append(paragraph)

separator = CharacterTextSplitter(chunk_size=10000, chunk_overlap=2000, separator=".")
documents = separator.split_documents(document_list)

print("Número total de documentos:", len(documents))

Número total de documentos: 120


In [5]:
batch_size = 10
sleep_time = 30

os.environ["CHROMA_DB_PATH"] = "C:\\Users\\BlueShift\\Documents\\Projects\\helpy_bot\\db\\chroma.sqlite3"

db = Chroma()

for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    print("Processando lote de documentos:", i, "a", min(i+batch_size, len(documents)))
    try:
        db.index_documents(batch, embeddings)
    except Exception as e:
        print("Erro ao processar o lote de documentos:", e)
    time.sleep(sleep_time)

Processando lote de documentos: 0 a 10
Erro ao processar o lote de documentos: 'Chroma' object has no attribute 'index_documents'
Processando lote de documentos: 10 a 20
Erro ao processar o lote de documentos: 'Chroma' object has no attribute 'index_documents'
Processando lote de documentos: 20 a 30
Erro ao processar o lote de documentos: 'Chroma' object has no attribute 'index_documents'
Processando lote de documentos: 30 a 40
Erro ao processar o lote de documentos: 'Chroma' object has no attribute 'index_documents'
Processando lote de documentos: 40 a 50
Erro ao processar o lote de documentos: 'Chroma' object has no attribute 'index_documents'
Processando lote de documentos: 50 a 60
Erro ao processar o lote de documentos: 'Chroma' object has no attribute 'index_documents'
Processando lote de documentos: 60 a 70
Erro ao processar o lote de documentos: 'Chroma' object has no attribute 'index_documents'


KeyboardInterrupt: 